In [14]:
import cv2
import numpy as np
import json
from PIL import Image
import json
from time import sleep
import subprocess


In [15]:
def processar_imagens(image1, image2, numeroframe):
    # Compute absolute difference between images
    diff = cv2.absdiff(image1, image2)
    # Convert to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)[1]

    # Find contours
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a monochrome image to draw the contours on
    monochrome = np.zeros_like(image1)

    # Create a GeoJSON with the contour information
    data = {
        'type': 'FeatureCollection',
        'features': []
    }

    # Set a tolerance to eliminate small objects
    tolerancia_opencv = 1

    # Iterate through contours
    for c in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(c)

        # If the area is above the tolerance
        if area > tolerancia_opencv:
            # Draw the contour on the monochrome image
            cv2.drawContours(monochrome, [c], -1, (255, 255, 255), -1)

            # Calculate the centroid of the contour
            M = cv2.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])

            # Create a GeoJSON feature
            feature = {
                'type': 'Feature',
                'properties': {},
                'geometry': {
                    'type': 'Point',
                    'coordinates': [cx, cy]
                }
            }
            # Append the feature to the data
            data['features'].append(feature)

            # Save the monochrome image
            cv2.imwrite(f'resultados/monochrome{str(numeroframe)}.jpg', monochrome)

            # Save the GeoJSON
            with open(f'resultados/centro_{str(numeroframe)}.json', 'w') as f:
                json.dump(data, f)

    # carregar imagem, converter, pixels para 2D array
    tolerancia = 25
    imagem_mono = Image.open(f'resultados/monochrome{str(numeroframe)}.jpg')
    def fn(x): return 255 if x > tolerancia else 0
    imagemRedim = imagem_mono.convert('L').point(fn, mode='1')
    pixels = list(imagemRedim.getdata())

    # criar lista de pontos, GeoJSON
    point_list = []
    for x in range(imagemRedim.width):
        for y in range(imagemRedim.height):
            valorPx = pixels[y * imagemRedim.width + x]
            # salvar apenas valores diferentes
            if valorPx > 0 :
                point = {
                    'type': 'Feature',
                    'geometry': {
                        'type': 'Point',
                        'coordinates': [x, y]},
                    'properties': {
                        'valorPx': valorPx
                        #'data': datetime.datetime.now().strftime("%Y-%m-%d"),
                        #'hora': datetime.datetime.now().strftime("%H:%M:%S")
                    }
                }
                point_list.append(point)

    if len(point_list) > 10:
        # JSON
        arquivoJson = {
            'type': 'FeatureCollection',
            'features': point_list
        }

    # salvar arquivo
    with open(f'resultados/contorno_{numeroframe}.json', 'w') as arquivo:
        json.dump(arquivoJson, arquivo)


In [16]:
#testar posicionamento

subprocess.run(
        ["libcamera-still", "--width", "640", "--height", "480", "--mode", "1640:1232", "-o", "imgs/inicio.jpg"])

Made DRM preview window
[0:32:29.946001275] [2942]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70
[0:32:30.006483423] [2943]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media2 and ISP device /dev/media0
[0:32:30.007663942] [2942]  INFO Camera camera.cpp:1028 configuring streams: (0) 1296x972-YUV420
[0:32:30.008141441] [2943]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1296x972-SGBRG10_1X10 - Selected unicam format: 1296x972-pGAA
Stream configuration adjusted
[0:32:35.179101989] [2942]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-YUV420 (1) 2592x1944-SGBRG10_CSI2P
[0:32:35.181409903] [2943]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 2592x1944-SGBRG10_1X10 - Selected unicam format: 2592x1944-pGAA
Still capture image received


AttributeError: module 'numpy' has no attribute 'float64'

In [ ]:
# tirar foto

def tirarFoto(arquivo):
    subprocess.run(
        ["libcamera-still", "--width", "320", "--height", "240", "--mode", "1640:1232", "-o", arquivo])


In [ ]:
numeroframe = 0
while numeroframe < 10:   
    tirarFoto('imgs/01.jpg')
    tirarFoto('imgs/02.jpg')
    image1 = cv2.imread('imgs/01.jpg')
    image2 = cv2.imread('imgs/02.jpg')
    processar_imagens(image1, image2, numeroframe)
    numeroframe += 1
    sleep(1)